<p align="center">
  <span style="font-size:22px"><b>ISEL</b></span><br>
  <span style="font-size:22px"><b>Departamento de Engenharia Informática</b></span><br>
  <span style="font-size:22px"><b>Licenciatura em Engenharia Informática e Multimédia</b></span><br>
  <span style="font-size:22px"><b>Processamento de Sinais Multimédia</b></span><br><br>
  <span style="font-size:18px"><b>1º Trabalho Prático</b></span><br>
  <span style="font-size:16px">3º Semestre 2025/2026</span><br><br>
  <span style="font-size:16px">Docente: Prof. Tiago Gonçalves</span><br><br>
  <span style="font-size:16px">Trabalho realizado por:</span><br>
  <span style="font-size:16px">David Santos nº51417</span><br>
  <span style="font-size:16px">Bernardo Aguiar nº52483</span><br>
  <span style="font-size:16px">Diogo Costa nº52453</span><br>
</p>

##### a) Implemente os Comb-filters na sua estrutura Feadback Comb-Filter FBCF(α, Dc) = (z^(−Dc))/(1−αz−Dc), onde α está associado ao ganho e Dc ao atraso introduzido (em nº de amostras).

i) Determine e mostre a resposta em frequência (amplitude e fase). Apresente o Diagrama-Polos-Zeros. Nota: Resolva para conjunto de ganhos separadamente.


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.signal as ss

Fs = 44100

def FBCF(alpha, atraso):
    Dc = int(atraso * (Fs / 1000))

    bks = np.zeros(Dc + 1)
    bks[Dc] = 1.0

    aks = np.zeros(Dc + 1)
    aks[0] = 1.0
    aks[Dc] = -alpha

    w, H = ss.freqz(bks, aks, worN=8192)
    f = w * Fs / (2 * np.pi)

    plt.figure(figsize=(12, 5))
    plt.subplot(1, 2, 1)
    plt.plot(f, np.abs(H))
    plt.title(f'Resposta em Amplitude do FBCF (alpha={alpha}, Dc={Dc})')
    plt.grid(True)
    plt.xlim(0, 500)
    plt.xlabel('Frequência (Hz)')

    plt.subplot(1, 2, 2)
    plt.plot(f, np.unwrap(np.angle(H)))
    plt.title('Resposta em Fase do FBCF')
    plt.xlabel('Frequência (Hz)')
    plt.ylabel('Fase (radianos)')
    plt.grid(True)
    plt.xlim(0, Fs/2)
    
    
    zeros, polos, k = ss.tf2zpk(bks, aks)
    
    # O sistema não possui zeros e possui um número de polos igual ao número de amostras (Dc)
    plt.figure(figsize=(12, 5))
    plt.subplot(1, 2, 1)
    plt.plot(np.real(zeros), np.imag(zeros), 'o', markersize=5, color='blue', label='Zeros')    
    plt.title('Diagrama de Polos e Zeros')
    plt.xlabel('Eixo Real')
    plt.ylabel('Eixo Imaginário')
    plt.xlim([-1.5, 1.5])
    plt.ylim([-1.5, 1.5])
    plt.grid(True)
    plt.legend()
    
    plt.subplot(1, 2, 2)
    plt.plot(np.real(polos), np.imag(polos), 'x', markersize=5, color='red', label='Polos')
    plt.title('Diagrama de Polos e Zeros')
    plt.xlabel('Eixo Real')
    plt.ylabel('Eixo Imaginário')
    plt.xlim([-1.5, 1.5])
    plt.ylim([-1.5, 1.5])
    plt.grid(True)
    plt.legend()

    plt.show()

#FBCF(0.3, 30)
FBCF(0.5, 40)
#FBCF(0.9, 45)
#FBCF(0.4, 35)

ii) Determine experimentalmente a resposta impulsional. Ilustre graficamente, guarde num ficheiro wave e escute o resultado.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import signal
from scipy.io import wavfile 

atraso = 40  
alpha = 0.8
Dc = int(atraso * (Fs / 1000))         

# Coeficientes do Numerador B
B = np.zeros(Dc + 1)
B[Dc] = 1.0

# Coeficientes do Denominador A
A = np.zeros(Dc + 1)
A[0] = 1.0
A[Dc] = -alpha

DURACAO_SEG = 1.0 
NUM_AMOSTRAS = int(Fs * DURACAO_SEG)

impulso = np.zeros(NUM_AMOSTRAS)
impulso[0] = 1.0

h_n = ss.lfilter(B, A, impulso)

tempo = np.arange(NUM_AMOSTRAS) / Fs

plt.figure(figsize=(10, 4))
plt.plot(tempo * 1000, h_n) 
plt.title(f'Resposta Impulsional do FBCF (alpha={alpha}, Dc={Dc})')
plt.xlabel('Tempo (ms)')
plt.ylabel('Amplitude')
plt.grid(True)
plt.xlim(0, 1000) 
plt.show()

NOME_FICHEIRO = 'fbcf_resposta_impulsional.wav'

h_n_normalizado = h_n / np.max(np.abs(h_n)) 

wavfile.write(NOME_FICHEIRO, Fs, h_n_normalizado.astype(np.float32))

print(f"\nResposta Impulsional guardada como: {NOME_FICHEIRO}")
print("Escute o resultado para identificar a sequência de ecos.")

iii) Aplique este filtro a sinais de teste (que considere adequados) e sinais audio para testar a sua funcionalidade.

In [ ]:
import numpy as np
from scipy import signal
from scipy.io import wavfile

B = np.zeros(Dc + 1)
B[Dc] = 1.0

A = np.zeros(Dc + 1)
A[0] = 1.0
A[Dc] = -alpha

DURACAO_SEG = 2 
FREQUENCIA_HZ = 440
AMPLITUDE = 0.8
FADE_TIME = 0.05 

NUM_AMOSTRAS = int(Fs * DURACAO_SEG)
tempo = np.arange(NUM_AMOSTRAS) / Fs

sinal_teste = AMPLITUDE * np.sin(2 * np.pi * FREQUENCIA_HZ * tempo) 

fade_samples = int(Fs * FADE_TIME)
fade_in = np.linspace(0, 1, fade_samples)
fade_out = np.linspace(1, 0, fade_samples)

sinal_teste[:fade_samples] *= fade_in
sinal_teste[-fade_samples:] *= fade_out

sinal_processado = signal.lfilter(B, A, sinal_teste) #Filtro aplicado aqui

sinal_processado_normalizado = sinal_processado / np.max(np.abs(sinal_processado))

wavfile.write('fbcf_audio_processado.wav', Fs, sinal_processado_normalizado.astype(np.float32))

print(f"Sinal de teste (tom {FREQUENCIA_HZ} Hz) gerado e processado com o FBCF.")
print(f"Sinal guardado como: fbcf_audio_processado.wav")